In [1]:
##########################################
# Cleaning Code                          
# This code is specifically designed to clean and process the historical FF book equity dataframe retrieved from the Ken French data website
# The code was separated from the main Python code to simplify the processes, such that the main code can directly import a cleaned dataset with minimal adjustments required
# The code is designed to produce a dataset which contains the gvkey and permno columns that will be used for merging with the CRSP and CCM datasets
##########################################

In [2]:
# Import the modules for processing the dataset

import pandas as pd
import numpy as np
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import math
from datetime import date

In [3]:
# Create a function which will be used to standardise the year in the dataset to the format of YYYY-MM-DD using the last day of each year (31st of December)
# In the cleaned dataset, there will be a yearend column which will represent the last day of each year (since the original data only had the year value and did not have day and month)

class MyCustomDate(date):
    def __new__(cls, year, month=12, day=31):
        return super().__new__(cls, year, month, day)

In [4]:
# Retrieve the data from a downloaded txt file from the Ken French website and convert into Pandas dataframe

data = []
with open("DFF_BE_With_Nonindust.txt", "r") as file:
    for line in file:
        try:
            row = line.strip().rstrip().split(" ")
            data.append(row)
        except Exception as e:
            print(f"Error processing line: {line.strip()}. Error: {e}")

historical_data = pd.DataFrame(data)

In [5]:
# Rename the important columns which we will keep
historical_data = historical_data.rename({0: 'CRSP_Permno', 1: 'First_Moody_Year', 2: 'Last_Moody_Year'}, axis='columns')

# Convert the dataframe into numeric format (float)
historical_data = historical_data.apply(pd.to_numeric)

# Replace nulls with NaNs
historical_data = historical_data.fillna(value=np.nan)

# Replace meaningless values (represented as -99.99) with NaNs
historical_data = historical_data.replace(-99.99, np.NaN)

# Drop all columns which only have NaN values
historical_data = historical_data.dropna(axis=1, how='all')

In [6]:
# Get all of the book equity values for each stock (one stock per row in the file)
# Each book equity value corresponds to a certain year, and each stock has a different starting year depending on when it was listed (with the earliest year being 1926)

bm_values = []

# Read each row
for i in range(0, len(historical_data)):
    row_ls = historical_data.values[i]
    # Get each value of the row stored into a new list
    bm_row = []
    for j in range(0, len(historical_data.values[i] - 1)):
        if not math.isnan(historical_data.values[i][j]):
               bm_row.append(historical_data.values[i][j])
    # bm_values is a list of lists, with each list inside containing all the book equity values of a single stock
    bm_values.append(bm_row)

In [7]:
##########################################
# Cleaning Code Part 2                   
# The code below performs the following:
# Step 1: Read the 'permno_gvkey_dictionary.npy' which was created in the main code, this dictionary has the gvkey corresponding to all the stock permnos
# Step 2: Create a dataframe which will be used to store the cleaned values for the historical book equity from the Ken French data
# Step 3: Keep a tracker which counts the number of times that the stock's book equity value is recorded on the Ken French website data, which starts with 1 and increments each time a new value for book equity is observed
# Step 4: Get the stock id (permno), first year and last year for each stock from the bm_values list created above
# Step 5: If a gvkey exists for a specific permno (retrieved using the permno_gvkey dictionary), get the corresponding gvkey, otherwise set this value to NaN
# Step 6: Starting from the first book value observed for each stock (in index position 3 in the list from above), get the book equity value and make the year corresponding to the book equity equal to the first year shifted by the number of counts (ie years observed), which should start from the first year and increase to the last year
# Step 7: Using the MyCustomDate function, assign a YYYY-MM-DD to each year corresponding to the last day of each year
# Step 8: Assign an additional value jdate which corresponds to the 6 months forward date of the previous year (eg for 1926-12-31, the jdate is 1927-06-30)
# Step 9: Append the values of gvkey, permno, date, yearend date, jdate, book value of equity and count to the new dataframe
# Step 10: Increment the count, and then convert the dataframe into a standard Pandas dataframe with column names 'gvkey', 'permno', 'datadate', 'yearend', 'jdate', 'be', 'count'
# Step 11: Sort the values by permno first and then by datadate
##########################################

In [8]:
# Load the 'permno_gvkey_dictionary.npy' created from the ff3_vF.ipynb code using the CCM block

permno_gvkey_dictionary = np.load('permno_gvkey_dictionary.npy',allow_pickle='TRUE').item()

#A new dataframe is created which stores the gvkey as an additional column alongside the columns from the section above
df_with_gvkey = []

# Perform the eleven steps mentioned above to create the dataframe
# Create this new dataframe, with gvkey as the left most column
for i in range(0, len(bm_values)):
    # Start with count = 1 to record the first observation of a book equity value for a certain stock
    count = 1
    # Get the stock id as permno
    stock_id = int(bm_values[i][0])
    # Get the gvkey corresponding to the permno
    if stock_id in permno_gvkey_dictionary:
        gvkey = permno_gvkey_dictionary[stock_id]
    # Get first and last year for each stock
    first_year = bm_values[i][1] - 1
    last_year = bm_values[i][2] - 1
    # Start looping through each stock's book equity values to append to a dataframe using the steps mentioned above
    for j in range(3, len(bm_values[i])):
        my_ls = []
        # Get one value per row for the dataframe such that stocks with multiple book equity values will have multiple rows in the dataframe
        bm_val = bm_values[i][j]
        year = int(first_year + j - 3)
        # Custom date function to present the date as YYYY-12-31
        custom_date = MyCustomDate(year)
        # Add 6 months to the custom date
        six_months_later = custom_date + relativedelta(months=+6)
        datadate = date(custom_date.year, custom_date.month, custom_date.day)
        datadate_six_months = date(six_months_later.year, six_months_later.month, six_months_later.day)
        # Append everything to the dataframe
        if gvkey:
            my_ls.append(gvkey)
        else:
            my_ls.append(np.nan)
        my_ls.append(stock_id)
        my_ls.append(datadate.isoformat())
        my_ls.append(datadate.isoformat())
        my_ls.append(datadate_six_months.isoformat())
        my_ls.append(bm_val)
        my_ls.append(count)
        # This is a new row that has been added to the dataframe
        df_with_gvkey.append(my_ls)
        # Increment the count
        count += 1
# Convert into standard Pandas dataframe
df_with_gvkey = pd.DataFrame(df_with_gvkey)
# Rename the columns
df_with_gvkey = df_with_gvkey.rename({0: 'gvkey', 1: 'permno', 2: 'datadate', 3: 'yearend', 4: 'jdate', 5: 'be', 6: 'count'}, axis='columns')
# Sort by permno and datadate
df_with_gvkey = df_with_gvkey.sort_values(by=['permno','datadate'])

In [9]:
# Output the newly created dataframe containing all of the historical book equity values from 1926 to a csv file
# This csv file will be loaded into the main code to simplify the processes, allowing for the separation of the cleaning step with the data processing step

df_with_gvkey.to_csv('DFF_BE_With_Nonindust_Cleaned_with_gvkey.csv')